In [35]:
import pandas as pd
import polars as pl
from tqdm import tqdm
from deep_translator import GoogleTranslator

In [21]:
df1 = pd.read_csv('./data/scraped_data_part1.csv')
df2 = pd.read_csv('./data/scraped_data_part2.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [22]:
translator = GoogleTranslator(source='cs', target='en')
col_map = {}
for col in df.columns:
    col_map[col] = translator.translate(col).lower().replace(' ', '_')
    
df = df.rename(columns=col_map)

In [23]:
df

,year,active,interrupted,completed,terminated_without_graduation,for_study_reasons,termination_of_accreditation,for_disciplinary_reasons,death,otherwise,college,faculty,study_program,year_of_enrollment
0,2002,127,0,0,0,0,0,0,0,0,11000 Univerzita Karlova,11110 1. lékařská fakulta,B0000 bakalářské studijní programy,2002
1,2003,107,3,0,17,16,0,0,1,0,11000 Univerzita Karlova,11110 1. lékařská fakulta,B0000 bakalářské studijní programy,2002
2,2004,96,2,0,12,12,0,0,0,0,11000 Univerzita Karlova,11110 1. lékařská fakulta,B0000 bakalářské studijní programy,2002
3,2005,54,1,42,1,1,0,0,0,0,11000 Univerzita Karlova,11110 1. lékařská fakulta,B0000 bakalářské studijní programy,2002
4,2006,10,0,44,1,1,0,0,0,0,11000 Univerzita Karlova,11110 1. lékařská fakulta,B0000 bakalářské studijní programy,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511048,2021,53,6,87,13,13,0,0,0,0,"7V000 Vysoká škola ekonomie a managementu, a.s.",7V900 celoškolská pracoviště,N6208 Ekonomika a management,2019
511049,2022,25,7,16,11,10,0,0,0,1,"7V000 Vysoká škola ekonomie a managementu, a.s.",7V900 celoškolská pracoviště,N6208 Ekonomika a management,2019
511050,2020,19,0,0,1,1,0,0,0,0,"7V000 Vysoká škola ekonomie a managementu, a.s.",7V900 celoškolská pracoviště,N6208 Ekonomika a management,2020
511051,2021,14,1,3,1,1,0,0,0,0,"7V000 Vysoká škola ekonomie a managementu, a.s.",7V900 celoškolská pracoviště,N6208 Ekonomika a management,2020


In [24]:
for col in ["college", "faculty", "study_program"]:
    print("Translating:", col)
    categories = df[col].unique()
    categories_map = {}
    for cat in categories:
        categories_map[cat] = translator.translate(cat)

    df[col+"_en"] = df[col].map(categories_map)
    df[col+"_cs"] = df[col]
    df.drop(columns=[col], inplace=True)

Translating: college
Translating: faculty
Translating: study_program


KeyboardInterrupt: 

In [28]:
col = "study_program"
categories = df[col].unique()
translated = list(categories_map.keys())
print(len(translated), len(categories))


(3986, 5059)

In [30]:
categories_left = [cat for cat in categories if cat not in translated]
for cat in tqdm(categories_left):
    categories_map[cat] = translator.translate(cat)

100%|██████████| 1073/1073 [09:43<00:00,  1.84it/s]


In [31]:
df[col+"_en"] = df[col].map(categories_map)
df[col+"_cs"] = df[col]


In [32]:
df.drop(columns=[col], inplace=True)

In [33]:
df.to_csv('./data/scraped_data_en.csv', index=False)

In [39]:
df = pl.read_csv("./data/scraped_data_en.csv")
df = df.select(
    [
        "college_en",
        "college_cs",
        "faculty_en",
        "faculty_cs",
        "study_program_en",
        "study_program_cs",
        "year_of_enrollment",
        "year",
        "active",
        "interrupted",
        "completed",
        "terminated_without_graduation",
        "for_study_reasons",
        "termination_of_accreditation",
        "for_disciplinary_reasons",
        "death",
        "otherwise",
    ]
)
df.write_csv('./data/scraped_data.csv')